In [10]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import io
import os, cv2
from PIL import Image
from PIL import ImageFile
#from rembg.bg import remove

In [11]:
mtcnn = MTCNN(keep_all=True, device='cuda')
new_img_dir = '/opt/ml/input/data/train/new_images'
img_path = '/opt/ml/input/data/train/images'
cnt = 0

In [23]:
for paths in os.listdir(img_path):
    if paths[0] == '.': continue
    
    sub_dir = os.path.join(img_path, paths)
    new_sub_dir = os.path.join(new_img_dir, paths)
    if not os.path.exists(new_sub_dir):  
        os.makedirs(new_sub_dir)
    
    for imgs in os.listdir(sub_dir):
        if imgs[0] == '.': continue
        
        img_dir = os.path.join(sub_dir, imgs)
        img = Image.open(img_dir)
        boxes,probs = mtcnn.detect(img)
        
        
        if not isinstance(boxes, np.ndarray):
            img = img.crop((50, 100, 350, 400))
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img.crop((xmin, ymin, xmax, ymax))
                
        tmp = os.path.join(new_img_dir, paths)
        img = img.resize((384,512))
#         cnt += 1
        img.save(os.path.join(tmp, imgs))
        